In [1]:
from functions import *
from simulation import *
import matplotlib.pyplot as plt
opt1 = EuropeanCallOption(100, 100, 1, 0.05, 0.2)

In [2]:
import torch 
import numpy as np
epsilons = torch.tensor([0.1])
eps = np.array([0.1])
mkt_order(eps, 10, 10, 10)

array([42])

In [13]:
def option_simulation(V, S, T, dt, K, r, sigma_daily):
    # options: a list of option objects
    # V: Cholesky factor of the covariance matrix
    # S: stock price path
    # T: Time of total trading days
    # dt: Time step in days.

    # K is np array of strike prices
    # r is the risk-free rate
    # sigma_daily is the daily volatility of the underlying asset

    # initialize the option prices, delta, and gamma
    N = int(T / dt)
    n = len(K)

    option_prices = np.zeros((N, n))
    delta = np.zeros((N, n))
    gamma = np.zeros((N, n))

    for i in range(N):
        for j in range(n):
            option = EuropeanCallOption(S[i], K[j], T - i * dt, r, sigma_daily)
            option_prices[i, j] = option.price()
            delta[i, j] = option.delta()
            gamma[i, j] = option.gamma()

    # add shock 
    shock = shock_path(V, T, dt)
    option_prices += shock

    return option_prices, delta, gamma

stock_path = stock_price_path(100, 0.05, 1, 0.01)
# generate the Cholesky factor of the covariance matrix
V = np.array([[1, 0.5], [0.5, 1]])   
price, delta, gamma = option_simulation(V, stock_path, 1, 0.01, np.array([100, 110]), 0.05, 0.2)
print(price.shape)

(100, 2)


In [37]:
# define a neural network for the policy 
class Net(torch.nn.Module):
    def __init__(self, n):
        self.n = 2
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(2 + 2 * n, 10)
        self.fc2 = torch.nn.Linear(10, 2 * n)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        # add relu to make sure the output is positive
        x = torch.relu(x)
        return x
    

def policy_test(t, Q, P, S):
    # Q is dim n numpy array
    # P is dim n numpy array
    # S is dim 1 numpy array
    
    net = Net(n)
    # use this network to output action 
    x = torch.tensor(np.concatenate([t, Q, P, S])).float()
    action = net(x)
    return action


def second_period_trading(policy, t, Q, P, S, dt, A, kappa):
    # policy: a function that takes in t, Q, P, S and returns the action
    # t: current time
    # Q: current inventory (dim: n)
    # P: current price (dim: n)
    # S: current shock (dim: 1)

    # get the action
    action = policy(t, Q, P, S) # dim: 2n

    # the action is a vector of bid and ask for each options 
    # for each option, we need to calculate the number of market orders 
    # let the first n be the bid and the second n be the ask
    n = int(len(action) / 2)
    bid_spread = action[:n]
    ask_spread = action[n:]

    # calculate the number of market orders
    # call the mkt_order function for each option
    # the result is a vector of number of market orders for each option
    # dim: 2n
    buy_orders = [mkt_order(epsilon.detach().numpy(), dt, A, kappa) for epsilon in bid_spread]
    sell_orders = [mkt_order(epsilon.detach().numpy(), dt, A, kappa) for epsilon in ask_spread]

    buy_orders = np.array(buy_orders)
    sell_orders = np.array(sell_orders)

    return buy_orders, sell_orders

def entire_trading(policy, P, S, dt, A, kappa):
    # P is sequence of option prices (dim: N x n)
    # S is the sequence of stock (dim: N)
    # n means the number of options
    # N means the total trading days

    # initialize the inventory
    Q = np.zeros(P.shape[1])

    inventory_path = np.zeros((P.shape[0], P.shape[1]))
    buy_order_path = np.zeros((P.shape[0], P.shape[1]))
    sell_order_path = np.zeros((P.shape[0], P.shape[1]))

    for i in range(P.shape[0]):
        curr_time = np.array([i * dt])  
        curr_P = np.array(P[i])
        curr_S = np.array([S[i]])

        buy_orders, sell_orders = second_period_trading(policy, curr_time, Q, curr_P, curr_S, dt, A, kappa)
        Q += buy_orders - sell_orders
        inventory_path[i] = Q
        buy_order_path[i] = buy_orders
        sell_order_path[i] = sell_orders

    return inventory_path, buy_order_path, sell_order_path

stock_path = stock_price_path(100, 0.05, 1, 0.01)
# generate the Cholesky factor of the covariance matrix
V = np.array([[1, 0.5], [0.5, 1]])
price, delta, gamma = option_simulation(V, stock_path, 1, 0.01, np.array([100, 110]), 0.05, 0.2)

# the following function is to compute the inventory path under policy
inventory_path, buy_order_path, sell_order_path = entire_trading(policy_test, price, stock_path, 0.01, 100, 3)

print(inventory_path.shape)
print(buy_order_path.shape)
print(sell_order_path.shape)

(100, 2)
(100, 2)
(100, 2)


In [ ]:
# generate 2 by 2 matrix
V = np.array([[2, 0.1], [0.1, 1]])

shock = shock_path(V, 1, 0.01)
shock.shape

(100, 2)